In [1]:
import os
import matplotlib.pyplot as plt
import pandas
import numpy
import matplotlib
import pandas as pd
from matplotlib.axis import Axis
from matplotlib.figure import Figure
from matplotlib.patches import Rectangle

from env import *
from visualiser import *
from main import *
from constants import *
# import xlsxwriter
import csv
%matplotlib inline

pygame 2.1.3.dev8 (SDL 2.0.22, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
SENSOR_RANGE = 10

n=range(2, 11)
ENV_SIZE=500
XL=250
YL=75

GOAL_X=250
GOAL_Y=425
MAX_T=50000

In [3]:
def calcDF(n):
    return [(SENSOR_RANGE+ROBOT_RADIUS)*(i-(n-1)/2) for i in range(n)] * n, [(SENSOR_RANGE+ROBOT_RADIUS)*(i-(n-1)/2) for i in range(n)for _ in range(n) ]

In [4]:
def plot_trajectory(env, fig:Figure = None, ax:Axis=None):

	if fig is None and ax is None:
		fig=matplotlib.pyplot.figure(figsize=(16, 16))
	if ax is None:
		ax=fig.add_subplot()
	# ax.invert_yaxis()

	posei=env.pose_history[:, env.N, :]
	ax.plot(posei[:env.t-1,0], posei[:env.t-1, 1], linestyle='dashdot',  label='Virtual Leader', color='blue', zorder=3)
	line=[None]*env.N
	for i in range(env.N):
					posei=env.pose_history[:, i, :]
					deadi=env.dead_history[:, i]
					posei=posei[~deadi]
					ax.plot(posei[:,0], posei[:, 1], label='Agent '+str(i), color='aqua')
	for wall in env.walls:
					ax.add_patch(Rectangle((wall.from_x, wall.from_y), wall.length_x, wall.length_y, color='black'))
	ax.scatter(env.xL0, env.yL0, marker='*', zorder=3, color='blue', s=150)
	ax.scatter(env.xG, env.yG, marker='*', zorder=3, color='blue', s=150)
	for agent in env.agents:
					ax.scatter(env.xL0+agent.dx, env.yL0+agent.dy, color='limegreen')
					ax.scatter(env.pose_history[env.t-1, agent.id, 0], env.pose_history[env.t-1, agent.id, 1], color='red' if agent.is_dead else 'limegreen')
	return fig


In [5]:
directory = '../run_exp_10'
if not os.path.exists(directory):
	os.makedirs(directory)
df=pandas.DataFrame(columns=['id','sensor_range' , 'n1', 'n2', 'n3', 'N', 'lived_agents', 'leader_goal_distance', 't', 'w1', 'w2', 'w3' ])
df.set_index('id', inplace=True)

In [6]:
n=10

In [7]:
DX,DY=calcDF(n)
env = Env(
        width=ENV_SIZE, height=ENV_SIZE, goal_x=GOAL_X, goal_y=GOAL_Y, N=n ** 2,
        desired_X=DX, desired_Y=DY, sensor_range=SENSOR_RANGE, leader_x=XL, leader_y=YL, robot_radius=ROBOT_RADIUS,
        sensor_detection_count=SENSOR_DETECTION_COUNT, buffer_size=MAX_T
    )
env.addObstacle(200, 200, 300, 300)


In [8]:
episode_gui(env, 5, 1, 1)